<a href="https://colab.research.google.com/github/TheRedPill-exe/Proyecto_fundamentos_basicos_de_python/blob/master/parcial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sodapy

### Pasando el código de la primera sección a colab
Módulo API

In [4]:
import pandas as pd
from sodapy import Socrata

def get_api_client():
    """
    Creates and returns a Socrata API client.
    """
    return Socrata("www.datos.gov.co", None)

def build_query(department):
    """
    Constructs the SoQL query to filter data by department.
    """
    return {"$where": f"departamento_nom='{department.upper()}'"}

def fetch_raw_data(client, query_params, record_limit):
    """
    Fetches raw data from the API using the given client, query parameters, and record limit.
    """
    try:
        return client.get("gt2j-8ykr", limit=record_limit, **query_params)
    except Exception as e:
        print(f"❌ Error fetching data from API: {e}")
        return []

def process_data(results):
    """
    Processes the raw API response into a structured Pandas DataFrame with only required columns.
    """
    if not results:
        print("⚠ No data returned from the API.")
        return []

    df = pd.DataFrame.from_records(results)

    # Define required columns
    required_columns = [
        "ciudad_municipio_nom",
        "departamento_nom",
        "edad",
        "fuente_tipo_contagio",
        "estado",
    ]

    # Add "pais_viajo_1_nom" if missing
    if "pais_viajo_1_nom" not in df.columns:
        df["pais_viajo_1_nom"] = "N/A"

    required_columns.append("pais_viajo_1_nom")

    # Keep only the columns that are available
    df = df[[col for col in required_columns if col in df.columns]]

    return df.to_dict(orient="records")

def fetch_data(department, record_limit):
    """
    Orchestrates the data fetching process from API to structured dictionary output.
    """
    client = get_api_client()
    query_params = build_query(department)
    raw_data = fetch_raw_data(client, query_params, record_limit)
    return process_data(raw_data)

Módulo UI

In [5]:
def get_user_input():
    """
    Asks the user for the department and the number of records to retrieve.
    """
    department = input("Enter the department name: ")

    while True:
        try:
            record_limit = int(input("Enter the number of records to retrieve: "))
            if record_limit > 0:
                break
            else:
                print("The number of records must be greater than 0.")
        except ValueError:
            print("Please enter a valid number.")

    return department, record_limit


def display_results(data):
    """
    Displays the retrieved data in a formatted table.
    """
    if not data:
        print("No data found for the given query.")
        return

    print("\nQuery Results:\n")
    print(f"{'City':<20} {'Department':<15} {'Age':<5} {'Type':<20} {'Status':<15} ")
    print("-" * 75)

    for record in data:
        print(f"{record.get('ciudad_municipio_nom', 'N/A'):<20} "
              f"{record.get('departamento_nom', 'N/A'):<15} "
              f"{record.get('edad', 'N/A'):<5} "
              f"{record.get('fuente_tipo_contagio', 'N/A'):<20} "
              f"{record.get('estado', 'N/A'):<15} ")

Módulo principal (anterior)

In [7]:
def main():
    #callig the function get_user_input
    department, record_limit = get_user_input()
    #calling the function fetch_data
    data = fetch_data(department, record_limit)
    #calling the function display_results
    display_results(data)



if __name__ == "__main__":
    main()

Enter the department name: risaralda
Enter the number of records to retrieve: 10



Query Results:

City                 Department      Age   Type                 Status          
---------------------------------------------------------------------------
BELEN DE UMBRIA      RISARALDA       81    Comunitaria          Leve            
SANTA ROSA DE CABAL  RISARALDA       64    Comunitaria          Leve            
SANTA ROSA DE CABAL  RISARALDA       40    Comunitaria          Leve            
PEREIRA              RISARALDA       20    Comunitaria          Leve            
SANTA ROSA DE CABAL  RISARALDA       20    Comunitaria          Leve            
PEREIRA              RISARALDA       49    Comunitaria          Leve            
DOSQUEBRADAS         RISARALDA       34    Comunitaria          Leve            
PEREIRA              RISARALDA       17    Comunitaria          Leve            
PEREIRA              RISARALDA       46    Comunitaria          Leve            
PEREIRA              RISARALDA       47    Comunitaria          Leve            


### Segunda parte
Aplicando los procedimientos de las paginas 138 hasta la 141 del libro.